# Code Audit

A full overview of the codebase, evaluating the workflow and systems. Problematic functions are highlighted and alternatives are proposed

## Utils
The different utility functions, and why they exist

In [4]:
from context import geomapi
import geomapi.utils as ut
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### `def get_min_average_and_max_value()`

In [3]:
# Useless function, nowhere used, you can just get the min, max and average separately
ut.get_min_average_and_max_value([[1,2,3]])

AttributeError: 'list' object has no attribute 'flatten'

### `def get_rotation_matrix_from_forward_up()`

Define a rotation based on the forward and up vector

In [12]:
print(ut.get_rotation_matrix_from_forward_up(np.array([0,0,1]), np.array([0,1,0])))
print(ut.get_rotation_matrix_from_forward_up(np.array([0,0,-1]), np.array([0,1,0])))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]


### `def convert_to_homogeneous_3d_coordinates()`

- why always return a 2D array?
- what if already homogeneous, but not 1 at the end? -> normalise

In [20]:
print(ut.convert_to_homogeneous_3d_coordinates([[1,1,5,1],[1,2,3,2]]))
print(ut.convert_to_homogeneous_3d_coordinates([1,1,5]))

[[1.  1.  5.  1. ]
 [0.5 1.  1.5 1. ]]
[[1. 1. 5. 1.]]


### `def get_geomapi_classes()`

What is the use for this?

In [ ]:
# What is the use for this?
ut.get_geomapi_classes()

[rdflib.term.URIRef('https://w3id.org/geomapi#Node'),
 rdflib.term.URIRef('https://w3id.org/geomapi#SetNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#PointCloudNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#MeshNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#LineSetNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#BIMNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#ImageNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#OrthoNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#PanoNode'),
 rdflib.term.URIRef('https://w3id.org/geomapi#RelativePart'),
 rdflib.term.URIRef('https://w3id.org/geomapi#AbsolutePart'),
 rdflib.term.URIRef('https://w3id.org/geomapi#Analysis'),
 rdflib.term.URIRef('https://w3id.org/geomapi#Result')]

### `def get_folder()`

2 functions exist

In [ ]:
print(ut.get_folder(r"C:\Users\jelle\Documents\DoctoraatLocal\geomapi\examples\Code_audit.ipynb"))
print(ut.get_folder_path(r"C:\Users\jelle\Documents\DoctoraatLocal\geomapi\examples\Code_audit.ipynb")) # deleted

C:\Users\jelle\Documents\DoctoraatLocal\geomapi\examples
C:\Users\jelle\Documents\DoctoraatLocal\geomapi\examples


### `def get_timestamp()`

The timestamp of a file should be when it is created, not when it is modified
this works on windows, but not on linux, since there is no build in function for that

In [16]:
print(ut.get_timestamp(r"C:\Users\jelle\Documents\DoctoraatLocal\geomapi\examples\Code_audit.ipynb"))
print(ut.get_timestamp("string"))

2025-03-12T14:08:33


ValueError: Path does not exist

### `def literal_to_python()`

Literals are pretty much always strings
to python only converts them to float or int, to number seems more appropriate

In [ ]:
print(ut.literal_to_python("10"))

1.0
1.0
10


### `def literal_to_list()`

In [51]:
print(ut.string_to_list("[1,2,3]")[0])
print(ut.literal_to_list("[5,ei,5.0]"))
print(ut.literal_to_list(5.0))

1
[5, 'ei', 5.0]
[5.0]


### `def cartesianTransform_to_literal()` 'featured3d_to_literal()'
This is obsolete

### `def validate_timestamp()`

In [ ]:
import dateutil.parser
dateutil.parser.parse("Tue Dec  7 09:38:13 2021")

datetime.datetime(2021, 12, 7, 9, 38, 13)

In [ ]:
# Beter om default datetime terug te geven, zo kan de precisie ook bewaard blijven
from datetime import datetime
#string
print(ut.validate_timestamp("2022:03:13 13:55:26", asStr=False),"2022-03-13T13:55:26")
#string
print(ut.validate_timestamp('Tue Dec  7 09:38:13 2021'),"2021-12-07T09:38:13")
#string
print(ut.validate_timestamp("1648468136.033126", millies=True),"2022-03-28T11:48:56.033126")
#datetime object
print(ut.validate_timestamp(datetime(2022,3,13,13,55,26)),"2022-03-13T13:55:26")


2022-03-13 13:55:26 2022-03-13T13:55:26
2021-12-07T09:38:13 2021-12-07T09:38:13
2022-03-28T11:48:56.033126 2022-03-28T11:48:56.033126
2022-03-13T13:55:26 2022-03-13T13:55:26


## `def check_if_subject_is_in_graph()`

In [26]:
from rdflib import Graph
imgGraphPath= r"C:\Users\jelle\Documents\DoctoraatLocal\geomapi\tests\testfiles\graphs\mesh_graph.ttl"
imgGraph=Graph().parse(str(imgGraphPath))
for s in imgGraph.subjects():
    print(s)

http://meshes#parking
http://meshes#parking
http://meshes#road
http://meshes#railway
http://meshes#road
http://meshes#railway
http://meshes#parking
http://meshes#parking
http://meshes#railway
http://meshes#parking
http://meshes#railway
http://meshes#parking
http://meshes#road
http://meshes#parking
http://meshes#parking
http://meshes#railway
http://meshes#parking
http://meshes#parking
http://meshes#road
http://meshes#road
http://meshes#road
http://meshes#railway
http://meshes#railway
http://meshes#parking
http://meshes#parking
http://meshes#road
http://meshes#road
http://meshes#railway
http://meshes#road
http://meshes#road
http://meshes#railway
http://meshes#road


### `def get_xsd_datatypes()` `get_ifcopenshell_class_name()`

nut hiervan?

## CADUtils

## Geometry Utils

In [27]:
from geomapi.utils import geometryutils as gmu
import quaternion
# Test with parameters (center, extent, euler_angles)
parameters = np.array([
    [4, 5, 6],  # Center
    [1, 2, 3],  # Extent
    [0, np.pi, 0]   # Euler angles (no rotation)
])
obb = gmu.get_oriented_bounding_box(parameters)
print(obb.R)
quaternion.as_euler_angles(quaternion.from_rotation_matrix(obb.R)) * 180/np.pi

[[-1.0000000e+00  0.0000000e+00  1.2246468e-16]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [-1.2246468e-16  0.0000000e+00 -1.0000000e+00]]


array([ 180.,  180., -180.])

In [ ]:
# going back and forth to eulers can change rotations, except for very basic ones like 90 deg
center = [0, 0, 0]
extent = [1, 2, 3]
euler_angles = [90, 0, 0]
euler_angles2 = [60, 80, 60]

obb = gmu.get_oriented_bounding_box(np.array([center, extent, euler_angles]), True)
obb2 = gmu.get_oriented_bounding_box(np.array([center, extent, euler_angles2]), True)
print(gmu.get_oriented_bounding_box_parameters(obb))
print(gmu.get_oriented_bounding_box_parameters(obb2))


The euler angles are derived from the rotation matrix, please note that this representation has a number of disadvantages
[ 0.  0.  0.  1.  2.  3. 90.  0.  0.]
The euler angles are derived from the rotation matrix, please note that this representation has a number of disadvantages
[  0.           0.           0.           1.           2.
   3.          84.23136778 -30.25159748  84.23136778]


In [ ]:
from geomapi.nodes import ImageNode

node = ImageNode()

node.world_to_pixel_coordinates([0,0,1])

array([[240.],
       [320.]])

## PanoNode

In [ ]:
_cartesianTransform = None
if _cartesianTransform is None:
    #you could initialize a pano in an upright position instead of forward to match a terrestrial vantage point
    rotation_matrix_90_x=np.array( [[ 1, 0, 0.],
                                    [ 0, 0,-1 ],
                                    [ 0, 1, 0 ]])  
    _cartesianTransform = gmu.get_cartesian_transform(rotation=rotation_matrix_90_x)    

## Tools

In [34]:
from geomapi import tools
import open3d as o3d
cloud = o3d.geometry.PointCloud()
tools.create_node(resource = cloud)

ValueError: Resource type not supported or len(resource.points) <3.